In [ ]:
'''Importing necessary libraries'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
'''reading data from csv'''

data = pd.read_csv('/content/drive/MyDrive/Problem Set 6/ps6_trainvalid.csv')

# Data Exploration and Preprocessing

In [ ]:
'''converting datetime column to datetime type and setting it as index'''

data.datetime = pd.to_datetime(data.datetime)
data.set_index(data.datetime, inplace=True)
data.drop('datetime', inplace=True, axis=1)
data

In [ ]:
'''checking for null values in the dataset'''

data.isnull().sum()

In [ ]:
'''checking distribution of numerical features'''

data.describe()

In [ ]:
'''visualizing distributions with histograms'''

plt.figure(figsize=(15,5))
plt.subplot(1,4,1)
plt.hist(data['temperature'], bins=25)
plt.title('Temperature')

plt.subplot(1,4,2)
plt.hist(data['humidity'], bins=25)
plt.title('Humidity')

plt.subplot(1,4,3)
plt.hist(data['pressure'], bins=50)
plt.title('Pressure')

plt.subplot(1,4,4)
plt.hist(data['wind_speed'], bins=50)
plt.title('Wind Speed')

In [ ]:
'''substituting null values with mean'''

temp_mean = data.temperature.mean()
humidity_mean = data.humidity.mean()
pressure_mean = data.pressure.mean()

data.temperature = data.temperature.fillna(temp_mean)
data.humidity = data.humidity.fillna(humidity_mean)
data.pressure = data.pressure.fillna(pressure_mean)

In [ ]:
'''filling null value in weather with most common weather type'''

data.weather = data.weather.fillna('sky is clear')
data.wind_direction = data.wind_direction.fillna(0.0)
data.wind_speed = data.wind_speed.fillna(0.0)

In [ ]:
'''since wind direction is in degrees, we split it into its components - sin and cos and create two respective columns'''

data['cos_wind'] = np.cos((data.wind_direction.values.reshape(len(data), 1)*np.pi)/180)
data['sin_wind'] = np.sin((data.wind_direction.values.reshape(len(data), 1)*np.pi)/180)

In [ ]:
'''checking final dataset for any null values'''

data.info()

In [ ]:
'''dropping wind_direction since it is no longer needed as we have sin and cos components for it'''

data = data.drop('wind_direction', axis=1)
data.head()

In [ ]:
# '''scaling numerical features into a range of -1 to +1 as a standardization technique'''

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(-1,1))
# data[['temperature','pressure','humidity','wind_speed']] = scaler.fit_transform(data[['temperature','pressure','humidity','wind_speed']])
# data.head()

In [ ]:
'''plotting temperature with respect to time'''

plt.figure(figsize=(25,5))
data['temperature'].plot()
plt.xlabel('Time')
plt.ylabel('Temperature')

In [ ]:
'''calculating rolling mean with a window of 12 hours'''

data['rolling_mean'] = data['temperature'].rolling(12).mean()
data['std_dev'] = data['temperature'].rolling(12).std()



---

Checking for stationarity of the timeseries

---



In [ ]:
'''plotting temperature along with its rolling mean'''


plt.figure(figsize=(25,5))
plt.plot(data.temperature[13:], 'b', label='original')
plt.plot(data.rolling_mean[13::500], 'k', label='rolling_mean', linewidth=4)
#plt.plot(data.std_dev[13::100], 'k', label='standard_deviation')
plt.legend(loc='best')
plt.xlabel('Time')
plt.ylabel('Temperature')

Looking at moving average, it does not look stationary, 
We confirm stationarity with Dickey Fuller Test

In [ ]:
'''implementing Dickey Fuller Test'''

res = adfuller(data['temperature'].dropna())    
studies = ['Test Statistic','p-value','#Lags Used','Number of Observations Used','Critical values']

In [ ]:
'''Print statistics from the Dickey Fuller Test'''

for value, label in zip(res, studies):
  print(label,": ",value)

Test-statistic < critical values therefore: stationary

p-value very less therefore: reject null hypothesis -> stationary



---

Plotting ACF and PACF plots
Referred from: https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/


---



In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(data['temperature'],lags=56,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(data['temperature'],lags=56,ax=ax2)

In [ ]:
# ACF and PACF 
from statsmodels.tsa.stattools import acf, pacf
lag_acf = acf(data.temperature, nlags=20)
lag_pacf = pacf(data.temperature, nlags=20, method='ols')
# ACF
plt.figure(figsize=(15,5))

plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(data.temperature)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(data.temperature)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

# PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(data.temperature)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(data.temperature)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

# ARIMA Model Implementation

In [ ]:
train_index = int(len(data.temperature)*0.8)
train_data = data.temperature[:train_index]
test_data = data.temperature[train_index:]
train_data.shape, test_data.shape

In [ ]:
'''order represents p, d and q values inferred from ACF (q) and PACF(p) plots'''

model = ARIMA(train_data, order=(2, 0, 1))
results_AR = model.fit(disp=-1)

In [ ]:
'''Calculating mean absolute error for predictions'''

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(results_AR.fittedvalues, train_data)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('MAE: %.4f'%mae)

In [ ]:
'''restoring the timeseries characteristics using SARIMA'''

model1=sm.tsa.statespace.SARIMAX(data['temperature'],order=(2, 0, 1),seasonal_order=(2,0,1,12))
final_results=model1.fit()

In [ ]:
'''forecasting for future values'''

data['forecast_seasonal']=final_results.predict(start=train_index,end=45013,dynamic=False)

plt.figure(figsize=(15,5))
plt.plot(data.temperature, label='original')
plt.plot(data.forecast_seasonal, label='prediction')
plt.legend(loc='best')

In [ ]:
forecast_len = len(data) - len(train_data)
forecast = results_AR.forecast(forecast_len, alpha=0.05)

In [ ]:
mae = np.mean(np.abs(forecast[0] - test_data))
mae